In [1]:
import numpy as np
import pandas as pd
from keras.models import load_model
import keras.backend as K
from sklearn.model_selection import KFold
from keras.layers import Input, Dense, Conv1D, Flatten, MaxPooling1D, Conv2D, MaxPooling2D, AveragePooling2D, Dropout, Reshape, normalization
from keras.models import Model
from keras.utils import to_categorical
from keras.layers.recurrent import LSTM
from sklearn import metrics
import random
from keras.models import model_from_json

Using TensorFlow backend.


In [2]:
def precision(y_true, y_pred):
    # Calculates the precision
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    # Calculates the recall
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def f1(test_Y, pre_test_y):
    """F1-score"""
    Precision = precision(test_Y, pre_test_y)
    Recall = recall(test_Y, pre_test_y)
    f1 = 2 * ((Precision * Recall) / (Precision + Recall + K.epsilon()))
    return f1 

def TP(test_Y,pre_test_y):
    #Calculates the number of true positive samples
    TP = K.sum(K.round(K.clip(test_Y * pre_test_y, 0, 1)))#TP
    return TP

def FN(test_Y,pre_test_y):
    #calculates the number of false negative samples
    TP = K.sum(K.round(K.clip(test_Y * pre_test_y, 0, 1)))#TP
    P=K.sum(K.round(K.clip(test_Y, 0, 1)))
    FN = P-TP #FN=P-TP
    return FN

def TN(test_Y,pre_test_y):
    #calculate the number of true negative samples
    TN=K.sum(K.round(K.clip((test_Y-K.ones_like(test_Y))*(pre_test_y-K.ones_like(pre_test_y)), 0, 1)))#TN
    return TN

def FP(test_Y,pre_test_y):
    #calculate the number of false positive samples
    N = (-1)*K.sum(K.round(K.clip(test_Y-K.ones_like(test_Y), -1, 0)))#N
    TN=K.sum(K.round(K.clip((test_Y-K.ones_like(test_Y))*(pre_test_y-K.ones_like(pre_test_y)), 0, 1)))#TN
    FP=N-TN
    return FP

data = np.array(pd.read_csv("allTestScale_infoGainSorted.csv"))
    
X1 = data[0:3089, 1:]#3089 is the number of positive samples;'1' is the label of positive sample
Y1 = data[0:3089, 0]#'0' is the label of negative sample
X2 = data[3089:, 1:]
Y2 = data[3089:, 0]
X = np.concatenate([X1, X2], 0)
Y = np.concatenate([Y1, Y2], 0)
#Y = Y.reshape((Y.shape[0], -1))
X = np.expand_dims(X, 2)
print(X)
print("X.shape: ", X.shape)
print("Y.shape: ", Y.shape)

model_name = '6fea_idx12Scale_infoGainSorted450.h5'
model_back = load_model(model_name,
                        custom_objects={'precision': precision,'recall':recall,'f1':f1,'TP':TP,'FN':FN,'TN':TN,'FP':FP})
# model = load_model('pcsf.h5')
accuracy = model_back.evaluate(X,Y)
# print 'loss', loss
print('accuracy', accuracy)
maxprobability = model_back.predict(X)
np.set_printoptions(threshold=np.inf)
print(maxprobability)
fw = open('Result.txt','w') #define result outputFile 
myprob = "\n".join(map(str, maxprobability[:, 0]))
fw.write(myprob)
predictclass = model_back.predict(X)
predictclass = np.argmax(predictclass,axis=1)
print(predictclass)

[[[0.157895]
  [0.619048]
  [0.360394]
  ...
  [0.535968]
  [0.300754]
  [0.529412]]

 [[0.157895]
  [0.380952]
  [0.461255]
  ...
  [0.209726]
  [0.332906]
  [0.529412]]

 [[0.368421]
  [0.380952]
  [0.461255]
  ...
  [0.465046]
  [0.313657]
  [0.      ]]

 ...

 [[0.210526]
  [0.47619 ]
  [0.360394]
  ...
  [0.753799]
  [0.394178]
  [0.      ]]

 [[0.631579]
  [0.285714]
  [0.623001]
  ...
  [0.246201]
  [0.264546]
  [0.735294]]

 [[0.105263]
  [0.190476]
  [0.343173]
  ...
  [0.212766]
  [0.263583]
  [0.529412]]]
X.shape:  (5978, 448, 1)
Y.shape:  (5978,)
5978/5978 [==============================] - 1s 138us/step
accuracy [0.16878664323934825, 0.7505854964256287, 0.5017015337944031, 0.3170377314090729, 0.3866664469242096, 10.07486629486084, 5.909090995788574, 13.91978645324707, 2.064171075820923]
[[0.62195504]
 [0.7154914 ]
 [0.5533864 ]
 [0.41623992]
 [0.7502458 ]
 [0.31424618]
 [0.19104418]
 [0.46172094]
 [0.7970283 ]
 [0.3725664 ]
 [0.30819434]
 [0.8434164 ]
 [0.6289548 ]
 [0.686

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 